# Imports

In [3]:
import pandas as pd; import seaborn as sb; import statsmodels.formula.api as smf; import pingouin;
from itertools import combinations; import os; from analysis_helpers import *; 
import scipy; from scipy import stats; import os; import math; import warnings
warnings.filterwarnings('ignore')

# Check data files

#### Check the number of participant directories in each experiment and group. 
(Naming convention: ##_YYYY_Mmm_DD or #_YYYY_Mmm_DD, depending on subject #). 


PRINT: # of participants per experiment.




#### Check that each participant has one file of each (below). 

- pres0.csv through pres8.csv
- mem0.csv through mem8.csv
- eye_data/##_0* through eye_data/##_8*
- for Group 2 participants: log files for each run

PRINT: subjects who have more or less than 1 of each file type.


In [6]:
%%bash

## declare array variables for exps and groups
declare -a exps=("sustained_attention_experiment" "variable_attention_experiment")
declare -a groups=("group1" "group2")

## for each exp
for e in "${exps[@]}"; do

    # for each group
    for g in "${groups[@]}"; do
        echo
        echo
        echo $e
        echo $g
        find ../$e/data/$g/*?_????_???_??* -maxdepth 0 -type d | wc -l
        echo
        echo
        # print the number of participant directories
        
        # each participant this group and experiment 
        for d in ../$e/data/$g/*  ; do


            # for each run (0-7)
            for n in {0..7}; do

                # count number of pres, mem, and gaze files this participant has
                pres=$(find $d/pres$n.csv | wc -l)
                mem=$(find $d/mem$n.csv | wc -l)
                eye=$(find $d/eye_data/*_$n* | wc -l)

                # if participant has more or less than 1 file for this run, print their info
                if [[ $pres -ne 1 ]] || [[ $mem -ne 1 ]] || [[ $eye -ne 1 ]] ; then
                    echo "$d"
                    echo "$n"
                    echo
                    echo "$pres"
                    echo "$mem"
                    echo "$eye"
                    echo
                fi
                
                # if group2, also count the number of log files
                if [ "$g" = "group2" ]; then
                    log=$(find $d/*$n*.log | wc -l)
                    
                    # if participant has more or less than 1 log file for this run, print their info
                    if [[ $log -ne 1 ]] && [[ $n -ne 1 ]] ; then
                        echo
                        echo "$d"
                        echo "$n"
                        echo "$log"
                        echo
                    fi
                    
                    # if participant has more or less than 1 log file for this run, print their info
                    if [[ $log -ne 2 ]] && [[ $n = 1 ]] ; then
                        echo
                        echo "$d"
                        echo "$n"
                        echo "$log"
                        echo
                    fi
                    
                fi

            done
        done
    done
done



sustained_attention_experiment
group1
      30


../sustained_attention_experiment/data/group1/31_2018_Oct_21
0

       1
       1
       0



sustained_attention_experiment
group2
      30



../sustained_attention_experiment/data/group2/7_2019_Oct_13
1
       1



variable_attention_experiment
group1
      30


../variable_attention_experiment/data/group1/0_2019_Feb_21
0

       1
       1
       0

../variable_attention_experiment/data/group1/12_2019_Mar_27
0

       1
       1
       0

../variable_attention_experiment/data/group1/1_2019_Feb_21
0

       1
       1
       0

../variable_attention_experiment/data/group1/25_2019_Apr_16
0

       1
       1
       0

../variable_attention_experiment/data/group1/5_2019_Feb_26
6

       1
       1
       0



variable_attention_experiment
group2
      25


../variable_attention_experiment/data/group2/10_2019_Nov_16
7

       1
       1
       0

../variable_attention_experiment/data/group2/28_2020_Jan_13
2

       1
       1
       0


find: ../sustained_attention_experiment/data/group1/31_2018_Oct_21/eye_data/*_0*: No such file or directory
find: ../variable_attention_experiment/data/group1/0_2019_Feb_21/eye_data/*_0*: No such file or directory
find: ../variable_attention_experiment/data/group1/12_2019_Mar_27/eye_data/*_0*: No such file or directory
find: ../variable_attention_experiment/data/group1/1_2019_Feb_21/eye_data/*_0*: No such file or directory
find: ../variable_attention_experiment/data/group1/25_2019_Apr_16/eye_data/*_0*: No such file or directory
find: ../variable_attention_experiment/data/group1/5_2019_Feb_26/eye_data/*_6*: No such file or directory
find: ../variable_attention_experiment/data/group2/10_2019_Nov_16/eye_data/*_7*: No such file or directory
find: ../variable_attention_experiment/data/group2/28_2020_Jan_13/eye_data/*_2*: No such file or directory
find: ../variable_attention_experiment/data/group2/29_2020_Jan_13/eye_data/*_7*: No such file or directory
find: ../variable_attention_experiment/

Five participants do not pass the file check - four are missing a single gaze data file and one does not have a log file for the practice run of the experiment, which does not affect our analysis. All participants completed the practice round, but there may have been an error or the experiment may have been restarted for this participant.